<a href="https://colab.research.google.com/github/kumarchandan/END/blob/main/week7/END_Week7_Assign_Sentiment_Analysis_Encoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
'''
sentiment_labels.txt contains all phrase ids and the corresponding sentiment labels, separated by a vertical line.
Note that you can recover the 5 classes by mapping the positivity probability using the following cut-offs:
[0, 0.2], (0.2, 0.4], (0.4, 0.6], (0.6, 0.8], (0.8, 1.0] 
for very negative, negative, neutral, positive, very positive, respectively. 
Please note that phrase ids and sentence ids are not the same.
'''
df_sentiment_labels = pd.read_csv('https://raw.githubusercontent.com/kumarchandan/END/main/week7/standford-sentiment-tree-bank/sentiment_labels.txt', sep='|')
df_sentiment_labels.tail()

,phrase ids,sentiment values
239227,239227,0.36111
239228,239228,0.38889
239229,239229,0.33333
239230,239230,0.88889
239231,239231,0.50000


- [0, 0.2], (0.2, 0.4], (0.4, 0.6], (0.6, 0.8], (0.8, 1.0] for 
- very negative, negative, neutral, positive, very positive,

In [3]:
col = 'sentiment values'
conditions = [
  (df_sentiment_labels[col] > 0) & (df_sentiment_labels[col] < 0.2),
  (df_sentiment_labels[col] >= 0.2) & (df_sentiment_labels[col] < 0.4),
  (df_sentiment_labels[col] >= 0.4) & (df_sentiment_labels[col] < 0.6),
  (df_sentiment_labels[col] >= 0.6) & (df_sentiment_labels[col] < 0.8),
  (df_sentiment_labels[col] >= 0.8) & (df_sentiment_labels[col] < 1.0),
]
choices = [1., 2., 3., 4., 5.]
df_sentiment_labels['sentiment_category'] = np.select(conditions, choices, default=1.0)

In [4]:
df_sentiment_labels.head()

,phrase ids,sentiment values,sentiment_category
0,0,0.50000,3.0
1,1,0.50000,3.0
2,2,0.44444,3.0
3,3,0.50000,3.0
4,4,0.42708,3.0


In [5]:
# dictionary.txt contains all phrases and their IDs, separated by a vertical line |
df_dictionary = pd.read_csv(
    'https://raw.githubusercontent.com/kumarchandan/END/main/week7/standford-sentiment-tree-bank/dictionary.txt',
    sep='|',
    header=None,
    names=['reviews', 'phrase ids']
  )
df_dictionary.tail()

,reviews,phrase ids
239227,zoning ordinances to protect your community fr...,220441
239228,zzzzzzzzz,179256
239229,élan,220442
239230,É,220443
239231,É um passatempo descompromissado,220444


In [6]:
df_dictionary['reviews'][df_dictionary['reviews'] == "The Rock is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash even greater than Arnold Schwarzenegger , Jean-Claud Van Damme or Steven Segal ."]

50444    The Rock is destined to be the 21st Century 's...
Name: reviews, dtype: object

In [7]:
df_sentiment_labels['phrase ids']

0              0
1              1
2              2
3              3
4              4
           ...  
239227    239227
239228    239228
239229    239229
239230    239230
239231    239231
Name: phrase ids, Length: 239232, dtype: int64

In [8]:
df = pd.merge(df_sentiment_labels, df_dictionary, on='phrase ids')
df.tail()

,phrase ids,sentiment values,sentiment_category,reviews
239227,239227,0.36111,2.0,your standard Hollywood bio-pic
239228,239228,0.38889,2.0,your typical ` fish out of water ' story
239229,239229,0.33333,2.0,zero .
239230,239230,0.88889,5.0,zippy jazzy score
239231,239231,0.50000,3.0,UNK


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 239232 entries, 0 to 239231
Data columns (total 4 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   phrase ids          239232 non-null  int64  
 1   sentiment values    239232 non-null  float64
 2   sentiment_category  239232 non-null  float64
 3   reviews             239232 non-null  object 
dtypes: float64(2), int64(1), object(1)
memory usage: 9.1+ MB


In [10]:
df['reviews'][df['reviews'] == "The Rock is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash even greater than Arnold Schwarzenegger , Jean-Claud Van Damme or Steven Segal ."]

226166    The Rock is destined to be the 21st Century 's...
Name: reviews, dtype: object

In [11]:
import random
import torch, torchtext
from torchtext import data

# manual seed
SEED = 42
torch.manual_seed(SEED) # manual_seed - Sets the seed for generating random numbers.


- ~Field.sequential – Whether the datatype represents sequential data. If False, no tokenization is applied. Default: True
- ~Field.include_lengths – Whether to return a tuple of a padded minibatch and a list containing the lengths of each examples, or just a padded minibatch. Default: False.


In [12]:
Review = data.Field(sequential=True, tokenize='spacy', batch_first=True, include_lengths=True)
Label = data.LabelField(tokenize='spacy', is_target=True, batch_first=True, sequential=False)

In [13]:
fields = [('reviews', Review), ('labels', Label)]
fields

[('reviews', <torchtext.data.field.Field at 0x7f7a5d346f28>),
 ('labels', <torchtext.data.field.LabelField at 0x7f7a5d354048>)]

Convert the data in dataframes to torchtext

In [14]:
print(df.reviews[1000])
print(df['reviews'][1000])
print(df.shape)

footnote
footnote
(239232, 4)


In [15]:
example = [data.Example.fromlist([df.reviews[i], df.sentiment_category[i]], fields) for i in range(df.shape[0])]

In [16]:
# Creating dataset
rotTomReviewDataset = data.Dataset(example, fields)

In [17]:
print(len(rotTomReviewDataset))

239232


In [18]:
vars(rotTomReviewDataset.examples[21000])

{'labels': 5.0,
 'reviews': ['is',
  'a',
  'vivid',
  ',',
  'vibrant',
  'individual',
  'and',
  'the',
  'movie',
  "'s",
  'focus',
  'upon',
  'her',
  'makes',
  'it',
  'successful',
  'and',
  'accessible',
  '.']}

Split into training, testing and validation sets by using split() method

In [19]:
(train, valid) = rotTomReviewDataset.split(split_ratio=[0.85, 0.17], random_state=random.seed(SEED))

In [20]:
(len(train), len(valid))

(199360, 39872)

In [21]:
vars(train.examples[10])

{'labels': 4.0,
 'reviews': ['vanishes',
  'somewhere',
  'between',
  'her',
  'hair',
  'and',
  'her',
  'lips',
  '.']}

Build vocabulary

In [22]:
Review.build_vocab(train)
Label.build_vocab(train)

In [23]:
print(vars(Review.vocab))
print(vars(Review.vocab).get('freqs').values())

{'freqs': Counter({'the': 63740, ',': 58945, 'a': 45709, 'of': 43467, 'and': 43250, '.': 31953, 'to': 30961, '-': 30193, "'s": 23554, 'is': 19117, 'in': 18902, 'that': 16557, 'it': 15224, 'as': 11556, 'with': 10325, 'for': 9938, 'its': 9422, 'film': 8791, 'movie': 8455, 'an': 7914, 'this': 7408, 'but': 6955, 'be': 6930, 'you': 6759, 'on': 6497, 'The': 6325, '`': 6279, "n't": 5822, 'by': 5670, "''": 5644, "'": 5492, 'more': 5240, 'his': 5220, 'not': 4943, 'than': 4816, 'about': 4736, 'one': 4673, 'at': 4671, 'like': 4638, '--': 4606, 'or': 4586, 'from': 4563, 'are': 4400, 'have': 4237, 'all': 4203, 'has': 3929, 'A': 3632, 'so': 3632, 'out': 3416, 'who': 3324, 'story': 3309, ')': 3263, 'up': 3152, 'too': 2992, 'most': 2978, 'good': 2936, 'into': 2920, '(': 2845, 'I': 2802, 'much': 2648, 'just': 2594, '...': 2578, 'time': 2556, 'their': 2528, 'characters': 2509, 'no': 2480, 'what': 2423, 'can': 2421, 'It': 2415, 'comedy': 2399, 'some': 2286, 'even': 2264, 'if': 2263, 'does': 2245, 'your':

In [24]:
print(vars(Label.vocab))

{'freqs': Counter({3.0: 99403, 4.0: 41780, 2.0: 35985, 5.0: 12652, 1.0: 9540}), 'itos': [3.0, 4.0, 2.0, 5.0, 1.0], 'stoi': defaultdict(<function _default_unk_index at 0x7f7a5d3790d0>, {3.0: 0, 4.0: 1, 2.0: 2, 5.0: 3, 1.0: 4}), 'vectors': None}


In [25]:
print('Size of review vocab : ', len(Review.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Review.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)
print('Reviews :', Review.vocab.stoi)

Size of review vocab :  20824
Size of label vocab :  5
Top 10 words appreared repeatedly : [('the', 63740), (',', 58945), ('a', 45709), ('of', 43467), ('and', 43250), ('.', 31953), ('to', 30961), ('-', 30193), ("'s", 23554), ('is', 19117)]
Labels :  defaultdict(<function _default_unk_index at 0x7f7a5d3790d0>, {3.0: 0, 4.0: 1, 2.0: 2, 5.0: 3, 1.0: 4})
Reviews : defaultdict(<function _default_unk_index at 0x7f7a5d3790d0>, {'<unk>': 0, '<pad>': 1, 'the': 2, ',': 3, 'a': 4, 'of': 5, 'and': 6, '.': 7, 'to': 8, '-': 9, "'s": 10, 'is': 11, 'in': 12, 'that': 13, 'it': 14, 'as': 15, 'with': 16, 'for': 17, 'its': 18, 'film': 19, 'movie': 20, 'an': 21, 'this': 22, 'but': 23, 'be': 24, 'you': 25, 'on': 26, 'The': 27, '`': 28, "n't": 29, 'by': 30, "''": 31, "'": 32, 'more': 33, 'his': 34, 'not': 35, 'than': 36, 'about': 37, 'one': 38, 'at': 39, 'like': 40, '--': 41, 'or': 42, 'from': 43, 'are': 44, 'have': 45, 'all': 46, 'has': 47, 'A': 48, 'so': 49, 'out': 50, 'who': 51, 'story': 52, ')': 53, 'up'

Divide the train set into batches
- BucketIterator will help us

In [26]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [27]:
train_iterator, valid_iterator = data.BucketIterator.splits(
    (train, valid),
    batch_size=32,
    sort_key=lambda x: len(x.reviews),
    sort_within_batch=True,
    device=device
)

Save the vocabulary for later use

In [28]:
import os, pickle
with open('tokenizer.pkl', 'wb') as tokens:
  pickle.dump(Review.vocab.stoi, tokens)

Build Model

- Use Embedding and LSTM modules for this classification task

- Network has:
  - 300 dimensional vector embedding
  - 2 stacked LSTMs with 100 hidden features
  - 1 FC layer to comopress the output of the LSTMs to our labels

In [29]:
import torch.nn as nn
import torch.nn.functional as F

class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
        
        super().__init__()          
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           batch_first=True)
        # try using nn.GRU or nn.RNN here and compare their performances
        # try bidirectional and compare their performances
        
        # Dense layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
      
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num directions]
        dense_outputs = self.fc(hidden)   
        
        # Final activation function softmax
        output = F.softmax(dense_outputs[0], dim=1)
            
        return output

In [30]:
# Define hyperparameters
size_of_vocab = len(Review.vocab)
embedding_dim = 300
num_hidden_nodes = 100
num_output_nodes = 5
num_layers = 2
dropout = 0.2

# Instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout = dropout)

In [31]:
print(model)

classifier(
  (embedding): Embedding(20824, 300)
  (encoder): LSTM(300, 100, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=100, out_features=5, bias=True)
)


In [32]:
# count number of params in the network
def count_parameters(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [33]:
print(f'Trainable parameters: {count_parameters(model):,}')

Trainable parameters: 6,489,305


Model training and evaluation

In [34]:
import torch.optim as optim

# optimzer and loss
optimizer = optim.Adam(model.parameters(), lr=2e-4)
criterion = nn.CrossEntropyLoss()

In [35]:
# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc

In [36]:
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

Training loop

In [37]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        review, review_lengths = batch.reviews   
        
        # convert to 1D tensor
        predictions = model(review, review_lengths).squeeze()  
        
        # compute the loss
        loss = criterion(predictions, batch.labels)        
        
        # compute the binary accuracy
        acc = binary_accuracy(predictions, batch.labels)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Evaluation loop

In [38]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            review, review_lengths = batch.reviews
            
            # convert to 1d tensor
            predictions = model(review, review_lengths).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.labels)
            acc = binary_accuracy(predictions, batch.labels)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Train and evaluate

In [39]:
N_EPOCHS = 10
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')

	Train Loss: 1.386 | Train Acc: 52.30%
	 Val. Loss: 1.342 |  Val. Acc: 55.73% 

	Train Loss: 1.314 | Train Acc: 58.77%
	 Val. Loss: 1.311 |  Val. Acc: 58.90% 

	Train Loss: 1.280 | Train Acc: 62.38%
	 Val. Loss: 1.294 |  Val. Acc: 60.71% 

	Train Loss: 1.257 | Train Acc: 64.70%
	 Val. Loss: 1.283 |  Val. Acc: 61.94% 

	Train Loss: 1.241 | Train Acc: 66.33%
	 Val. Loss: 1.279 |  Val. Acc: 62.32% 

	Train Loss: 1.230 | Train Acc: 67.54%
	 Val. Loss: 1.276 |  Val. Acc: 62.42% 

	Train Loss: 1.219 | Train Acc: 68.62%
	 Val. Loss: 1.274 |  Val. Acc: 62.79% 

	Train Loss: 1.211 | Train Acc: 69.52%
	 Val. Loss: 1.273 |  Val. Acc: 62.97% 

	Train Loss: 1.203 | Train Acc: 70.22%
	 Val. Loss: 1.274 |  Val. Acc: 62.85% 

	Train Loss: 1.197 | Train Acc: 70.85%
	 Val. Loss: 1.271 |  Val. Acc: 63.12% 



Test the model

In [40]:
#load weights and tokenizer

path='./saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('./tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

#inference 

import spacy
nlp = spacy.load('en')

def classify_review(review):
    # very negative, negative, neutral, positive, very positive
    categories = {
        1: "Very Negative",
        2: "Negative",
        3: "Neutral",
        4: "Positive",
        5: "Very Positive"
      }
    
    # tokenize the review 
    tokenized = [tok.text for tok in nlp.tokenizer(review)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction = model(tensor, length_tensor)

    _, pred = torch.max(prediction, 1) 
    
    return categories[pred.item()]

In [49]:
classify_review("this is not a good stuff")

'Negative'